### Importing relevant packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

from ast import literal_eval
from math import sqrt

In [2]:
steam_data = pd.read_csv("dataset/steam_app_data.csv")
steamspy_data = pd.read_csv("dataset/steamspy_data.csv")

## Handling Data
We will clean the data obtained from this [Kaggle Steam Store Raw Data](https://www.kaggle.com/datasets/nikdavis/steam-store-raw), which contains data collected until May 2019. We will merge relevant columns of `steam_app_data.csv` and `steamspy_data.csv`, dropping missing data and cleaning the data to form our usable dataset.
Most of the data is stored in dictionary format, eg. \[{'id': '23', 'description': 'Indie'}, {'id': '3', 'description': 'RPG'}\]. Hence to get the relevant information we will use multiple `get_` functions to parse the dictionaries, using `literal_eval` from `ast` to evaluate the strings as Python structures.


In [3]:
# get relevant columns from datasets
data = steam_data[["type", "name", "steam_appid", "controller_support", "dlc", "short_description", "demos", "platforms", "movies", "achievements", "release_date", "is_free", "genres", "categories"]]
spy_data = steamspy_data[["appid", "developer", "publisher", "positive", "negative", "owners", "average_forever", "median_forever", "initialprice", "languages", "tags"]]

spy_data = spy_data.rename(columns={"appid":"steam_appid"})
# merge datasets based on steam appid
data = data.merge(spy_data, how = "inner", on = "steam_appid")

# drop rows which are not games or with no name, duplicates (repeated steam appid)
data = data.dropna(subset = ["type", "name"]).drop_duplicates(subset = ["steam_appid"])
data = data.drop(["type"], axis = 1)
# drop rows of games that have been deleted
data = data[data["name"] != "none"]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29074 entries, 0 to 29232
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                29074 non-null  object 
 1   steam_appid         29074 non-null  int64  
 2   controller_support  5998 non-null   object 
 3   dlc                 4975 non-null   object 
 4   short_description   29050 non-null  object 
 5   demos               2138 non-null   object 
 6   platforms           29074 non-null  object 
 7   movies              27157 non-null  object 
 8   achievements        26842 non-null  object 
 9   release_date        29074 non-null  object 
 10  is_free             29074 non-null  object 
 11  genres              29028 non-null  object 
 12  categories          28510 non-null  object 
 13  developer           28960 non-null  object 
 14  publisher           28878 non-null  object 
 15  positive            29074 non-null  int64  
 16  nega

In [4]:
# drop rows without english language
data = data.dropna(subset = ["languages"])
data = data[data["languages"].str.contains("English")]
data["languages"] = data["languages"].str.replace(", ", ";")

In [5]:
# filling null values
variables = ["controller_support", "dlc", "demos", "movies"]

# 1 for exists, 0 for does not exist
for var in variables:
    data[var] = np.where(data[var].isnull(), 0, 1)

In [6]:
# cleaning price data

print("Number of free games:", data[data["is_free"]].shape[0])
print("Number of games without price:", data[data["initialprice"] == 0].shape[0])

# drop games which are not free but still have no price data
data = data.drop(data[(data["is_free"] == 0) & (data["initialprice"] == 0)].index)
data = data.drop(["is_free"], axis = 1)
data["initialprice"] = data["initialprice"] / 100

Number of free games: 2715
Number of games without price: 3483


In [7]:
# cleaning platforms data

def get_platforms(cell):
    # evaluate data stored as string as dictionary using ast.literal_eval
    platforms = literal_eval(cell)
    # generate ; separated list of supported platforms if platform is True
    return ';'.join(platform for platform in platforms.keys() if platforms[platform])

data["platforms"] = data["platforms"].copy().apply(get_platforms)

In [8]:
# cleaning achievement data

def get_achievements(cell):
    if cell is not np.nan:
        achievements = literal_eval(cell)
        return achievements["total"]
    else:
        return 0

data["achievements"] = data["achievements"].copy().apply(get_achievements)

In [9]:
# cleaning release date data

def get_release_date(cell):
    release_date = literal_eval(cell)
    if release_date["coming_soon"]:
        return np.nan
    elif release_date["date"] == '':
        return np.nan
    else: 
        return release_date["date"]

data["release_date"] = data["release_date"].copy().apply(get_release_date)

# drop unreleased games or games with no date
data = data.dropna(subset = ["release_date"])

In [10]:
import re

# convert release date to date time format
def format_date(cell):
    if re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', cell):
        # day mon, year => day mon year
        return cell.replace(',', '')
    elif re.search(r'[A-Za-z]{3} [\d]{4}', cell):
        # add 1st day of month if only month, year is given
        return '1 ' + cell

data["release_date"] = pd.to_datetime(data["release_date"].apply(format_date), format = "%d %b %Y", errors = "raise")
data = data.dropna(subset = ["release_date"])

In [11]:
# cleaning tags data

def get_tags(cell):
    if cell == '[]':
        return np.nan
    else:
        tags_dict = literal_eval(cell)
        for tag in tags_dict.keys():
            tags = ';'.join(tags_dict)
        return tags

data["tags"] = data["tags"].copy().apply(get_tags)

In [12]:
# clean genres data

def get_genres(cell):
    if cell is not np.nan:
        genres_dictlist = literal_eval(cell)
        genres_list = []
        for genres_dict in genres_dictlist:
            genres_list.append(genres_dict["description"])
        for genre in genres_list:
            genres = ';'.join(genres_list)
        return genres
    else:
        return np.nan
    
data["genres"] = data["genres"].copy().apply(get_genres)

In [13]:
# drop games with no developer data
data = data.dropna(subset = ["developer"])

# fill in missing publisher with developer name (self-published games?)
for i in data[data["publisher"].isnull()].index:
    data.loc[i, "publisher"] = data["developer"][i]

In [14]:
# add column for total number of positive + negative recommendations
data.insert(loc = 16, column = "recommendations", value = data["positive"] + data["negative"])

In [15]:
# get unique official steam genres
data = data.reset_index(drop = True)
steam_genres = set()
for i in range(data.shape[0]):
    if data["genres"][i] is np.nan:
        continue
    all_genres = data["genres"][i].split(";")
    for steam_genre in all_genres:
        steam_genres.add(steam_genre)
steam_genres

{'Accounting',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Audio Production',
 'Casual',
 'Design & Illustration',
 'Documentary',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Gore',
 'Indie',
 'Massively Multiplayer',
 'Nudity',
 'Photo Editing',
 'RPG',
 'Racing',
 'Sexual Content',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Tutorial',
 'Utilities',
 'Video Production',
 'Violent',
 'Web Publishing'}

In [16]:
# remove very very dirty data
data = data[(data["genres"].str.contains("Nudity") == False) & (data["genres"].str.contains("Sexual Content") == False) & (data["tags"].str.contains("Nudity") == False) & (data["tags"].str.contains("Sexual Content") == False)]

In [17]:
# drop games without genres data 
data = data.dropna(subset = ["genres"])
data = data.reset_index(drop = True)

In [18]:
# remove steam genres from tags, tags => additional tags
for i in range(data.shape[0]):
    if data["tags"][i] is not np.nan:
        non_steam_tags = list(set(data["tags"][i].split(sep = ';')) - set(steam_genres))
        if non_steam_tags:
            data.loc[i, "tags"] = ';'.join(non_steam_tags)
        else:
            data.loc[i, "tags"] = np.nan
data.rename(columns = {"tags" : "additional_tags"}, inplace = True)

In [19]:
# get multiplayer data

def get_multiplayer(cell):
    if cell is not np.nan:
        cat_dictlist = literal_eval(cell)
        for cat_dict in cat_dictlist:
            if (cat_dict["description"] == "Multi-player" or cat_dict["description"] == "Online Multi-Player" 
                or cat_dict["description"] == "Local Multi-Player" or cat_dict["description"] == "Co-op"):
                return 1
            elif (cat_dict["description"] == "Single-player"):
                return 0
    else:
        return np.nan
    
data["is_multiplayer"] = data["categories"].copy().apply(get_multiplayer)
# drop games without multiplayer data
data = data.dropna(subset = ["is_multiplayer"])
data = data.drop(["categories"], axis = 1)

## Calculating Rating of Game
We will use the recommendations by users (`positive` or `negative`) as a ratio to determine the rating of a game. 
- However, games do not have the same number of recommendations. eg. A game that has only 3 recommendations (3 positive : 0 negative) should not be considered as good as a game that has 10000 recommendations (10000 positive : 0 negative). 
- Furthermore, games that have low number of recommendations will be heavily skewed if we only use positive/total recommendations. eg. A game that has 3 recommendations (3 positive : 0 negative) may be rated higher than a game with 10000 recommendations (9999 positive : 1 negative)

Hence we need to balance the proportion of positive recommendations with the uncertainty of small sample size. We will hence use Wilson Score to take into account the sample size, where Wilson Confidence Interval considers binomial distribution for score calculation.

![Wilson Score Interval](https://miro.medium.com/max/634/0*fYYMk52egqm-9h55.png)
- p is the observed fraction of positive recommendations
- n is the total number of recommendations
- zα/2 is the (1-α/2) quantile of the standard normal distribution

In [20]:
# use wilson score as an a better estimate of positive:negative rating
# Wilson Confidence Interval considers binomial distribution for score calculation
def wilson_score(positive, negative, z):
    n = positive + negative

    if n == 0:
        return 0

    p = float(positive) / n

    left = p + 1/(2*n)*z*z
    right = z*sqrt(p*(1-p)/n + z*z/(4*n*n))
    under = 1+1/n*z*z

    return (left - right) / under

z = 1.96 # 95% confidence interval => z-score = 1.96
data["rating"] = data.apply(lambda row: round(wilson_score(row.positive, row.negative, 1.96)*100, 1), axis = 1)

In [21]:
# drop rows with no ratings
data = data[data["recommendations"] != 0]
data = data.drop(["positive", "negative", "recommendations"], axis = 1)
data = data.reset_index(drop = True)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25306 entries, 0 to 25305
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                25306 non-null  object        
 1   steam_appid         25306 non-null  int64         
 2   controller_support  25306 non-null  int32         
 3   dlc                 25306 non-null  int32         
 4   short_description   25292 non-null  object        
 5   demos               25306 non-null  int32         
 6   platforms           25306 non-null  object        
 7   movies              25306 non-null  int32         
 8   achievements        25306 non-null  int64         
 9   release_date        25306 non-null  datetime64[ns]
 10  genres              25306 non-null  object        
 11  developer           25306 non-null  object        
 12  publisher           25306 non-null  object        
 13  owners              25306 non-null  object    

> **name** : game's name  
> 
> **steam_appid** : Steam Application ID (unique)  
> 
> **controller_support** : whether game offers controller support
> 
> **dlc** : whether game offers downloadable content (DLC) 
> 
> **short_description** : short description of game 
> 
> **demos** : whether game had a demo 
> 
> **platforms** : supported platforms (from windows, mac, linux)
> 
> **movies** : whether game provides cinematics on steam page 
> 
> **achievements** : number of achievements in game  
> 
> **release_date** : release date of game
> 
> **genres** : comma separated list of genres
> 
> **developer** : comma separated list of the developers of the game  
> 
> **publisher** : comma separated list of the publishers of the game
> 
> **owners** : owners of this application on Steam as a range
> 
> **average_forever** : average playtime since March 2009. In minutes.
> 
> **median_forever** : median playtime since March 2009. In minutes.
> 
> **initialprice** : price on release in US dollars.
> 
> **languages** : comma separated list of supported languages
> 
> **tags** : comma separated list of user-provided tags
> 
> **is_multiplayer** : whether game is multiplayer, otherwise singleplayer
> 
> **ratings** : rating of a game based on user reviews

In [23]:
data[:]

,name,steam_appid,controller_support,dlc,short_description,demos,platforms,movies,achievements,release_date,...,developer,publisher,owners,average_forever,median_forever,initialprice,languages,additional_tags,is_multiplayer,rating
0,Counter-Strike,10,0,0,Play the world's number 1 online action game. ...,0,windows;mac;linux,0,0,2000-11-01,...,Valve,Valve,"10,000,000 .. 20,000,000",17612,317,9.99,English;French;German;Italian;Spanish - Spain;...,Old School;Survival;Assassin;e-sports;First-Pe...,1.0,97.3
1,Team Fortress Classic,20,0,0,One of the most popular online action games of...,0,windows;mac;linux,0,0,1999-04-01,...,Valve,Valve,"5,000,000 .. 10,000,000",277,62,4.99,English;French;German;Italian;Spanish - Spain;...,Old School;Fast-Paced;Class-Based;Co-op;Funny;...,1.0,82.8
2,Day of Defeat,30,0,0,Enlist in an intense brand of Axis vs. Allied ...,0,windows;mac;linux,0,0,2003-05-01,...,Valve,Valve,"5,000,000 .. 10,000,000",187,34,4.99,English;French;German;Italian;Spanish - Spain,Historical;Class-Based;Co-op;First-Person;Sing...,1.0,88.6
3,Deathmatch Classic,40,0,0,Enjoy fast-paced multiplayer gaming with Death...,0,windows;mac;linux,0,0,2001-06-01,...,Valve,Valve,"5,000,000 .. 10,000,000",258,184,4.99,English;French;German;Italian;Spanish - Spain;...,First-Person;Classic;Shooter;FPS;Arena Shooter...,1.0,80.7
4,Half-Life: Opposing Force,50,0,0,Return to the Black Mesa Research Facility as ...,0,windows;mac;linux,0,0,1999-11-01,...,Gearbox Software,Valve,"5,000,000 .. 10,000,000",624,415,4.99,English;French;German;Korean,Silent Protagonist;Moddable;Co-op;Singleplayer...,0.0,94.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25301,Room of Pandora,1065230,0,0,The Room of Pandora is a third-person interact...,0,windows,1,7,2019-04-24,...,SHEN JIAWEI,SHEN JIAWEI,"0 .. 20,000",0,0,2.99,English;Japanese;Simplified Chinese;Traditiona...,Puzzle,0.0,43.8
25302,Cyber Gun,1065570,0,0,Cyber Gun is a hardcore first-person shooter w...,0,windows,1,0,2019-04-23,...,Semyon Maximov,BekkerDev Studio,"0 .. 20,000",0,0,1.99,English,Cyberpunk;Fast-Paced;3D Platformer;First-Perso...,0.0,56.5
25303,Super Star Blast,1065650,1,0,Super Star Blast is a space based game with ch...,0,windows,1,24,2019-04-24,...,EntwicklerX,EntwicklerX,"0 .. 20,000",0,0,4.99,English,NaN,0.0,0.0
25304,New Yankee 7: Deer Hunters,1066700,0,0,Pursue a snow-white deer through an enchanted ...,0,windows;mac,1,0,2019-04-17,...,Yustas Game Studio,Alawar Entertainment,"0 .. 20,000",0,0,6.99,English;German;Korean;Russian,NaN,0.0,34.2


In [24]:
# export clean dataset
data.to_csv("steamdata_clean.csv", index = False)